# Introduction to Differential Equations

---

# Exercises

In [ ]:
%matplotlib inline
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy.linalg as lin
from scipy.io import loadmat
from mpl_toolkits.mplot3d import Axes3D

## 1. LIF model with time-varying input current

In our lecture example of the LIF model, we used a constant $I_{input}$ for simplicity and to allow for a tractable analytical solution. However, the problem becomes much more interesting when $I_{input}$ also varies with time, such that:

$$
C \frac{dV}{dt} = (E_\text{leak} - V) \frac{1}{R} + I(t)
$$

Assuming $RC = 20 ms$, $E_{leak} = -70 mV$, and $V_0 = -70 mV$, use Euler's method with stepsize $1 ms$ to model the above differential equation from $t = 0$ to $t = 1 s$ with the following input currents:

1. $I(t)$ is a step-function, where $I(t) = 1000C$ amps for $250$ ms $< t < 750$ ms and $I(t) = 0$ everywhere else.
2. $I(t) = I_0\sin(\omega t)$, where $I_0 = 1000C$ amps and angular frequency $\omega = 10$

**Note:** Don't think too hard about the $C$ in the input current functions -- this is just to "cancel out" the $C$ in the differential equation when it gets moved to the right side (i.e. just use ``1000`` and not ``1000*C`` in your code).

In [ ]:
# initialize constants

## 1. make a function for the input current
# step function
    
# sinusoid                  


## 2. function for membrane potential dynamics


## 3. apply Euler's method   
    
    
## 4. plot results
# include a plot for both the membrane potential and the input current


Next, plot slope fields for $\frac{dV}{dt}$ separately for each input current using the intervals $t = [0,1] s$ and $V = [-100, -30] mV$. Use 20 samples in each dimension.

In [ ]:
## 1. define sample points to compute the slopes


## 2. iterate over points to plot the slopes and create the slope field



---
## 2. Drift-diffusion model
Alongside studying neural circuitry, differential equations have also been used to study cognitive processes such as decision making. Specifically, tasks studying *evidence accumulation* use differential equations to model behavior, an example of which is known as the *Poisson clicks task*. In this task, rats are presented with differing numbers of auditory clicks from left and right speakers in a behavioral chamber. Following the click train presentation, the animal must choose the side with the greater number of clicks. Below is a figure representation of the task (Brunton, Botvinick, & Brody, *Science*, 2013).

<img src="pclicks.png" width=600>

The process that describes the animal's ability to infer the side with greater clicks (the process of evidence accumulation) can be modeled by a *stochastic differential equation*, also known as the *drift-diffusion model*. This model describes how an internal decision variable evolves over time in the presence of auditory impulses and various sources of noise. An influential version of the drift-diffusion model was introduced by Brunton, Botvinick, and Brody (2013), explicitly:

$$
\frac{da}{dt} = (\delta_{t,t_R} \cdot \eta_r \cdot C - \delta_{t,t_L} \cdot \eta_L \cdot C) + \lambda a
 + \sigma_a \frac{dW}{dt} $$

Solving the above eqation will give us the accumulator memory, $a$, which represents the estimate of the right versus left evidence accrued at some point in time. 

The first term, $(\delta_{t,t_R} \cdot \eta_r \cdot C - \delta_{t,t_L} \cdot \eta_L \cdot C)$, describes how left or right clicks change the accumulator memory, where
- $C$ is the click magnitude
- $\delta_{t,t_R}$ and $\delta_{t,t_L}$ are delta functions at the times of the right and left clicks
- $\eta_R$ and $\eta_L$ describe sensory noise that scales the magnitude of accumulated right or left clicks, which are independently drawn from $N(0,\sigma_s)$

A second differential equation describes sensory adaptation of subsequent clicks, impacting the click magnitude $C$ over time by:

$$
\frac{dC}{dt} = \frac{1-C}{\tau_{\phi}} + (\phi - 1)C(\delta_{t,t_R}+\delta_{t,t_L})
$$

$\phi$ parameterizes how $C$ is scaled after each click, where $\phi>1$ represents facilitation (i.e. the magnitude of $C$ grows with each click) and $\phi<1$ represents depression (i.e. the magnitude of $C$ shrinks with each click). $\tau_{\phi}$ is the time constant that dictates how quickly $C$ recoveres to it's unadapted value of $1$.

The second term, $\lambda a$, describes the consistent drift in the accumulator memory, where $\lambda$ parameterizes the rate (with time constant $\tau=\frac{1}{\lambda}$) and direction of the drift. When $\lambda < 0$, the accumulator memory is "forgetful" and drifts toward $a = 0$, meaning later clicks affect the decision more than earlier clicks. When $\lambda > 0$, the accumulator memory is "unstable" and drifts away from $a = 0$, where earlier clicks affect the decision more than later clicks.

The last term, $\sigma_a \frac{dW}{dt}$, describes the "diffusion" element of the model, which is the noise attributed to the accumulator estimate $a$. $\frac{dW}{dt}$ is a white-noise Wiener process, also referred to as brownian motion, scaled by $\sigma_a$.

A visual example of a solution is illustrated in box 1 of this study.

<img src="ddm.png" width=600>

The black trace represents the accumulator estimate $a$ over time, where the influence of parameters introduced above are pointed out on the plot. The value at the end of the trial represents the accumulator's belief in which side had more clicks, and its magnitude is proportional to its "confidence" in the choice. Additional model parameters include:
- $B$, the decision boundary, which describes the threshold of evidence needed to make a decision
- $bias$, which shifts the "neutral" evidence state toward the left or right decision boundary, representing an inherent bias towards one of the two sides

While $B$ and $bias$ are generally included in drift diffusion models, as well as a "lapse rate" parameter that parameterizes the fraction of trials on which the animal responds randomly, we'll be ignoring these for the purpose of this exercise.

### 2.1: Generating click trains

In this exercise, we'll be numerically solving for the accumulator variable $a$ using the above differential equation for randomly generated trains of right and left clicks. But before we can do that, we need to construct the input stimulus. Using details from Brunton's and colleagues' supplementary material, we'll construct a function that generates randomly distributed click stimuli for both "left" and "right" clicks. Specifically, this function should output two delta functions at the click times for right and left clicks, or $\delta_{t,t_R}$ and $\delta_{t,t_L}$ above. Delta functions are a can of worms that we won't go into for this exercise, so we'll simplify things by assuming these functions take the form:

$$\delta_R(t) = \left\{ 
  \begin{array}{ c l }
    \frac{1}{dt} & \quad \textrm{if } t = t_R \\
    0                 & \quad \textrm{otherwise}
  \end{array}
\right.$$

for right clicks, where $dt$ is the time step between stimulus samples and $t_R$ are the times of the right clicks. This will replace $\delta_{t,r_R}$ in the differential equation. Similarly for left clicks: 

$$\delta_L(t) = \left\{ 
  \begin{array}{ c l }
    \frac{1}{dt} & \quad \textrm{if } t = t_L \\
    0                 & \quad \textrm{otherwise}
  \end{array}
\right.$$

where $t_L$ are the times of the left clicks, replacing $\delta_{t,t_L}$.

Create this function with the following properties:
1. Use an input variable, ``stim_dur``, to indicate how long you want each stimulus or click train to last in seconds
2. Use a second input variable, ``dt``, to indicate the step size between stimulus samples. In other words, $1/dt$ is the sampling rate of the stimulus, meaning that the output click trains should be length ``stim_dur`` $/$ ``dt``
3. Draw the number of right clicks from $\text{Pois}(r_R)$ and the number of left clicks from $\text{Pois}(r_L)$, where $r_R$ and $r_L$ are the click rates for right and left clicks, respectively. These rates should be randomly drawn such that $r_L+r_R = 20$ clicks per second.
4. Randomly distrute the right and left clicks throughout the stimulus interval (i.e. choose random times within the interval of length ``stim_dur`` $/$ ``dt`` corresponding to each of the clicks)
5. Return two variables, ``delta_right`` and ``delta_left``, corresponding to $\delta_R$ and $\delta_L$ above


In [ ]:
## function for click trains

# get number of samples for click train based on stim_dur and dt
# to initialize delta_right and delta_left

# draw click rates and # of clicks

# randomly distribute click times & store in delta_right and delta_left

### 2.2: Ideal performance

Now that we have a function to generate click trains, we can start approximating the drift-diffusion model. To get an intuition for how each term affects the accumulator, we'll start with a simplified version of the model and then introduce the pieces back in. 

To simplify the model, we can assume conditions that invoke "ideal performance". This is achieved by setting:

$$
\lambda=0, \sigma_a^2 = \sigma_s^2 = 0, \frac{dC}{dt} = 0
$$

This reduces the accumulator to one that only pays attention to the left and right clicks, unaffected by noise, drift, or adaptation:

$$
\frac{da}{dt} = (\delta_{t,t_R} \cdot C - \delta_{t,t_L} \cdot  C)
$$

where $C$ stays at a constant value of 1.

Use Euler's method to approximate $a$ with initial condition $a(0) = 0$, and parameters $\text{stim_dur}=1s$, $dt=0.001s$ and plot your solution using randomly generated click trains from your function above.

In [ ]:
# initialize variables

## 1. Use Euler's method to approximate a
# get click trains from function

# for-loop for euler's method

## 2. Plot your solution

### 2.3: Adding in the drift

You may have noticed that the above solution looks a lot like a step function. Indeed, it perfectly counts the total number of left and right clicks at any point in time. But things in the real world aren't quite so perfect; sometimes we may start to forget or misremember the number of clicks we've accumulated. This is where the "drift" part of the drift-diffusion model comes into play.

Reintroducing the drift term gives us the equation:

$$
\frac{da}{dt} = (\delta_{t,t_R} \cdot C - \delta_{t,t_L} \cdot  C) + \lambda a
$$

Use Euler's method to approximate this new accumulator value with the drift term, using the same initial conditions and setting  $\lambda = -20$, and plot your solution. You can do this below or by editing your code above.

In [ ]:
# same idea as above but add in drift

Our negative value for $\lambda$ demonstrates a forgetful or "leaky" accumulator. We no longer see perfect steps, but instead see our accumulator decay in the absense of new clicks.

### 2.4: Adding in adaptation

As you'll learn in your core courses, oftentimes repeated presentations of a stimulus will lead to sensory adaptation. This can manifest as a *depression*, where sensory receptors decrease their sensitivity to the stimuli, or as a *facilitation*, where sensory receptors increase their sensitivity to the stimuli. This phenomenon is accounted for in the drift-diffusion model by having an additional differential equation describe the dynamics of the click magnitude $C$. 

To observe how adaptation of $C$ affects our accumulator, we'll take a step back before adding the drift term and use the equation

$$
\frac{da}{dt} = (\delta_{t,t_R} \cdot C - \delta_{t,t_L} \cdot  C)
$$

except instead of $C$ being set to a constant $1$, it will also be updated according to 

$$
\frac{dC}{dt} = \frac{1-C}{\tau_{\phi}} + (\phi - 1)C(\delta_{t,t_R}+\delta_{t,t_L})
$$

This turns our drift-diffusion model into a *system* of differential equations. Much like a single differential equation, Euler's method can be used to approximate a system of differential equations. The only difference is that you're updating the value of multiple variables (in this case two) instead of one. By editing your code above or by writing new code below, approximate the accumulator with click adaptiation and plot your results. 

In [ ]:
# same idea as above but add in adaptation

You may have noticed that the steps now vary in size; specifically, they're getting smaller with repeated clicks, which is consistent with depression when $\phi < 1$.

### 2.5: Adding in noise

Being forgetful isn't the only imperfection we need to account for. Similar to how you may *think* you feel your phone buzzing in your pocket when it actually didn't, the accumulator may *think* it heard a click when it actually didn't (or, conversely, miss a click that was actually played). Additionally, while we try our best for sound isolation, you can't completely eliminate irrelevant auditory stimuli, which can also interfere with your perception of the clicks you hear. Both of these are captured by adding noise to the model.

There are two sources of noice included in the drift-diffusion model. The first is the term $\sigma_a \frac{dW}{dt}$, which constitutes the "diffusion" part of the drift-diffusion model. $\frac{dW}{dt}$ denotes a white noise Weiner process, also known as Brownian motion or a "random walk". This noise can be thought of as general uncertainty in the accumulator value, akin to accidentally missing clicks or hearing clicks that didn't occur. The second is captured by the variables $\eta_r$ and $\eta_l$, which represents sensory noise that may interfere with the magnitude of right and left clicks.

These sources of noise are the last additions to the drift-diffusion model we'll visit in this exercise. To see the effect of noise on the "ideal" model, approximate the drift-diffusion model with only the sources of noise such that

$$
\frac{da}{dt} = (\delta_{t,t_R} \cdot \eta_r \cdot C - \delta_{t,t_L} \cdot \eta_L \cdot C) + \sigma_a \frac{dW}{dt} 
$$

holding $C=1$. Use the values $\sigma_a = 2$ and $\sigma_s = 0.1$.

**TIPS:**
- Without worring too much about the specifics of a Weiner process, you can use the following principle to compute it in Euler's method: $dW \sim \sqrt{dt} N(0,1)$
- Remember that $\eta_r$ and $\eta_l$ are independently drawn from $N(0,\sigma_s)$

In [ ]:
# same idea as above but add in noise

### 2.6: Putting it all together

Now that we've looked at the separate impact of each factor of the drift-diffusion model, let's finally put everything together and approximate a solution to the full model.

$$
\frac{da}{dt} = (\delta_{t,t_R} \cdot \eta_r \cdot C - \delta_{t,t_L} \cdot \eta_L \cdot C) + \lambda a
 + \sigma_a \frac{dW}{dt} 
$$

$$
\frac{dC}{dt} = \frac{1-C}{\tau_{\phi}} + (\phi - 1)C(\delta_{t,t_R}+\delta_{t,t_L})
$$

Let's use the same initial conditions and parameters we've used along the way: $\text{stim_dur} = 1, dt = 0.001, a(0) = 0, C(0) = 1, \lambda = -20, \tau_{\phi} = 0.05, \phi = 0.6, \sigma_a = 2, \sigma_s = 0.1$.

As a bonus, you can plot the solution to the full model along with solutions to the various iterations we've looked at above to see how they all differ!

In [ ]:
# initialize variables

# set model parameters

## 1. Use Euler's method to approximate a
# initialize a
# use separate a's for each version of the model to plot them together at the end

# get the right and left clicks
   
# for loop for euler's method 
    # calculate all model versions together in the same for loop using the same click trains

## 2. Plot results
